I've seen a lot of people talking about how SARS-CoV-2 would be exterminated if lockdowns were lifted. This seems to be based on an opinion by Knut Wittkowski, as can be seen here https://www.thecollegefix.com/epidemiologist-coronavirus-could-be-exterminated-if-lockdowns-were-lifted.

So I did the math.

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: "%.2f" % x)

The best data I could find about death rates after Googling for a minute is this https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm. It was sampled between February 12 - March 16, 2020. These numbers are optimistic for the scenario described by Knut Wittkowski (ex. no shutdowns/quarantine/shelter in place for those who are low risk), but it doesn't really matter. When you do the math, it still looks bad, even with these optimistic death rates.

In [2]:
death = {
    "Death Rate": {         
        "0 - 19": 0 / 100,
        "20 - 44": (0.1 + 0.2) / 2 / 100,
        "45 - 54": (0.5 + 0.8) / 2 / 100,
        "55 - 64": (1.4 + 2.6) / 2 / 100,
        "65 - 74": (2.7 + 4.9) / 2 / 100,
        "75 - 84": (4.3 + 10.5) / 2 / 100,
        "85 +": (10.4 + 27.3) / 2 / 100
    }
}
death_df = pd.DataFrame(death)
death_df

,Death Rate
0 - 19,0.00
20 - 44,0.00
45 - 54,0.01
55 - 64,0.02
65 - 74,0.04
75 - 84,0.07
85 +,0.19


The best data on population by age in the US that I could get after Googling for a minute is this https://data.census.gov/cedsci/table?q=us%20population%20by%20age.

In [3]:
pop = {
    "US Population": {        
        "0 - 4": 19646315,
        "5 - 9": 19805900,
        "10 - 14": 21392922,
        "15 - 19": 21445493,
        "20 - 24": 21717962,
        "25 - 29": 23320702,
        "30 - 34": 22023972,
        "35 - 39": 21571302,
        "40 - 44": 19927151,
        "45 - 49": 20733440,
        "50 - 54": 20871804,
        "55 - 59": 21624541,
        "60 - 64": 20662821,
        "65 - 69": 17107288,
        "70 - 74": 13464025,
        "75 - 79": 9378512,
        "80 - 84": 6169441,
        "85 +": 6303848
    }
}

I'm going to take this data and match the bins that were used by the CDC for the death rates (shown near the top).

In [4]:
pop = {
    "US Population":
    {
        "0 - 19": pop["US Population"]["0 - 4"] + pop["US Population"]["5 - 9"] + pop["US Population"]["10 - 14"] + pop["US Population"]["15 - 19"],
        "20 - 44": pop["US Population"]["20 - 24"] + pop["US Population"]["25 - 29"]+ pop["US Population"]["30 - 34"] + pop["US Population"]["35 - 39"] + pop["US Population"]["40 - 44"],
        "45 - 54": pop["US Population"]["45 - 49"] + pop["US Population"]["50 - 54"],
        "55 - 64": pop["US Population"]["55 - 59"] + pop["US Population"]["60 - 64"],
        "65 - 74": pop["US Population"]["65 - 69"] + pop["US Population"]["70 - 74"],
        "75 - 84": pop["US Population"]["75 - 79"] + pop["US Population"]["80 - 84"],
        "85 +": pop["US Population"]["85 +"]
    }
}

In [5]:
pop_df = pd.DataFrame(pop)
pop_df

,US Population
0 - 19,82290630
20 - 44,108561089
45 - 54,41605244
55 - 64,42287362
65 - 74,30571313
75 - 84,15547953
85 +,6303848


In [6]:
df = pd.concat([pop_df["US Population"], death_df["Death Rate"]], axis=1)

Let's take a look at how many people would die if various percentages of the total US population was infected by SARS-CoV-2.

In [7]:
percent_of_population_infected = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]

In [8]:
for x in percent_of_population_infected:
    df["Percent Infected {}".format(x)] = df["US Population"] * df["Death Rate"] * x
summary_df = df.append(df.agg(["sum"]))

In [9]:
summary_df

,US Population,Death Rate,Percent Infected 0.1,Percent Infected 0.2,Percent Infected 0.3,Percent Infected 0.4,Percent Infected 0.5,Percent Infected 0.6,Percent Infected 0.7,Percent Infected 0.8,Percent Infected 0.9,Percent Infected 1.0
0 - 19,82290630,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
20 - 44,108561089,0.00,16284.16,32568.33,48852.49,65136.65,81420.82,97704.98,113989.14,130273.31,146557.47,162841.63
45 - 54,41605244,0.01,27043.41,54086.82,81130.23,108173.63,135217.04,162260.45,189303.86,216347.27,243390.68,270434.09
55 - 64,42287362,0.02,84574.72,169149.45,253724.17,338298.90,422873.62,507448.34,592023.07,676597.79,761172.52,845747.24
65 - 74,30571313,0.04,116170.99,232341.98,348512.97,464683.96,580854.95,697025.94,813196.93,929367.92,1045538.90,1161709.89
75 - 84,15547953,0.07,115054.85,230109.70,345164.56,460219.41,575274.26,690329.11,805383.97,920438.82,1035493.67,1150548.52
85 +,6303848,0.19,118827.53,237655.07,356482.60,475310.14,594137.67,712965.21,831792.74,950620.28,1069447.81,1188275.35
sum,327167439,0.33,477955.67,955911.34,1433867.02,1911822.69,2389778.36,2867734.03,3345689.71,3823645.38,4301601.05,4779556.72


In [10]:
us_population = summary_df["US Population"]["sum"]
print(f"US Population: {us_population}")
print("Deaths if 10% of population is infected: {}".format(int(summary_df["Percent Infected 0.1"]["sum"])))
print("Deaths if 50% of population is infected: {}".format(int(summary_df["Percent Infected 0.5"]["sum"])))
print("Deaths if entire US population is infected: {}".format(int(summary_df["Percent Infected 1.0"]["sum"])))

US Population: 327167439
Deaths if 10% of population is infected: 477955
Deaths if 50% of population is infected: 2389778
Deaths if entire US population is infected: 4779556


Currently, the government is hoping that less than 10% of the U.S. population gets infected, so that can we keep the deaths below **478 thousand**. If everyone in the US was infected by SARS-CoV-2, about **4.78 million** people would die.

Okay, so let's do the math for what Knut Wittkowski is describing. Let's say his plan works perfectly, so everyone that is high risk never gets infected and never dies. He wants no social distancing, so that means everyone else does get infected.

In [11]:
age_groups = list(df.index)
number_of_age_groups = len(age_groups)
for i in range(1, number_of_age_groups):
    population_at_high_risk = df.iloc[-i:, df.columns.get_loc("US Population")].sum()
    percent_of_population_at_high_risk = population_at_high_risk / us_population
    deaths_if_x_percent_infected = {}
    for x in percent_of_population_infected:
        deaths_if_x_percent_infected["Percent Infected {}".format(x)] = df.iloc[:-i, df.columns.get_loc("Percent Infected {}".format(x))].sum()
    
    print("-" * 70)
    print("High risk age groups: {}".format(", ".join(age_groups[-i:])))
    print("US population at high risk: {}".format(population_at_high_risk))
    print("Percent of population at high risk: {}".format(percent_of_population_at_high_risk))
    print("Deaths if 10% of low risk population is infected: {}".format(deaths_if_x_percent_infected["Percent Infected 0.1"]))
    print("Deaths if 50% of low risk population is infected: {}".format(deaths_if_x_percent_infected["Percent Infected 0.5"]))
    print("Deaths if all of low risk population is infected: {}".format(deaths_if_x_percent_infected["Percent Infected 1.0"]))

----------------------------------------------------------------------
High risk age groups: 85 +
US population at high risk: 6303848
Percent of population at high risk: 0.01926795655236339
Deaths if 10% of low risk population is infected: 359128.13755000004
Deaths if 50% of low risk population is infected: 1795640.68775
Deaths if all of low risk population is infected: 3591281.3755
----------------------------------------------------------------------
High risk age groups: 75 - 84, 85 +
US population at high risk: 21851801
Percent of population at high risk: 0.06679087951658906
Deaths if 10% of low risk population is infected: 244073.28535000002
Deaths if 50% of low risk population is infected: 1220366.42675
Deaths if all of low risk population is infected: 2440732.8535
----------------------------------------------------------------------
High risk age groups: 65 - 74, 75 - 84, 85 +
US population at high risk: 52423114
Percent of population at high risk: 0.16023328654047386
Deaths if

Just read through some of the numbers that are output in the cell above to get a feel for the situation. To summarize, if we did what Knut Wittkowski is suggesting and decided that everyone above the age of 65 was high risk, then we'd have to **quarantine 16% of the US population and 1.28 million people die**.